In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
from langchain.vectorstores import Pinecone
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.embeddings import OpenAIEmbeddings
import pickle

In [3]:
# Load Data
loader = UnstructuredFileLoader("state_of_the_union.txt")
raw_documents = loader.load()
len(raw_documents)

1

In [4]:
# Split text
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(raw_documents)
len(documents)

11

In [5]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = "2023-06-01-preview" # # this version is required for annotations

model = os.environ['CHAT_MODEL_NAME']
embedding_model = os.environ['EMBEDDING_MODEL_NAME']

In [6]:
# Load Data to vectorstore
embeddings = OpenAIEmbeddings(chunk_size=1)
vectorstore = FAISS.from_documents(documents, embeddings)

In [7]:
# Save vectorstore
with open("vectorstore.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

In [8]:
from langchain.prompts.prompt import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains import RetrievalQA


from langchain.chat_models import AzureChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [9]:
def get_chain(vectorstore):
    chat = AzureChatOpenAI(temperature=0,
        max_tokens=500,
        openai_api_base=openai.api_base,
        openai_api_version=openai.api_version,
        deployment_name=model,
        openai_api_key=openai.api_key,
        openai_api_type = openai.api_type    
    )

    retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    from langchain.memory import ConversationBufferMemory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        retriever=retriever,
        memory=memory,
        chain_type="stuff"
    )
    return qa_chain

def ask_with_memory(vectorstore, question, chat_history=[]):
    chat = AzureChatOpenAI(temperature=0,
        max_tokens=500,
        openai_api_base=openai.api_base,
        openai_api_version=openai.api_version,
        deployment_name=model,
        openai_api_key=openai.api_key,
        openai_api_type = openai.api_type    
    )

    retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 3})

    # from langchain.memory import ConversationBufferMemory
    # memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=chat,
        retriever=retriever,
        chain_type="stuff",
        verbose=False
    )

    result = qa_chain({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    return result, chat_history
    



In [10]:
qa_chain = get_chain(vectorstore)
chat_history = []
print("Chat with your docs!")
while True:
    question = input()
    if question in ["quit", "exit"]:
        break
    result = qa_chain({"question": question, "chat_history": chat_history})
    chat_history.append((question, result["answer"]))
    print("Human:")
    print(question)
    print("AI:")
    print(result["answer"])

Chat with your docs!
Human:
Hello, how many sanctions were imposed on Russia?
AI:
The context does not provide a specific number of sanctions imposed on Russia. However, it does mention that the United States and its allies are enforcing powerful economic sanctions on Russia, cutting off its largest banks from the international financial system, preventing its central bank from defending the Russian Ruble, and choking off its access to technology. Additionally, the U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs, and the U.S. and its European allies are finding and seizing their ill-begotten gains.
Human:
What did President say about Ketanji Brown Jackson?
AI:
The United States and its allies are enforcing powerful economic sanctions on Russia. They are cutting off Russia’s largest banks from the international financial system, preventing Russia’s central bank from defending the Russian Ruble, and choking off Russia’s access t

In [11]:
chat_history = []
print("Chat with your docs!")
while True:
    question = input()
    if question in ["quit", "exit"]:
        break
    result, chat_history = ask_with_memory(vectorstore, question, chat_history)
    chat_history.append((question, result["answer"]))
    print("User:")
    print(question)
    print("AI:")
    print(result["answer"])
    

Chat with your docs!
User:
My name is Shiva! what is your name?
AI:
I'm an AI language model created by OpenAI. I don't have a name, but you can call me OpenAI. How can I assist you today, Shiva?
User:
What did the president say about Ketanji Brown Jackson?
AI:
The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson and mentioned that she is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers.
User:
How many sanctions were imposed on Russia?
AI:
The context does not provide a specific number of sanctions imposed on Russia. However, it mentions that the United States and its allies are enforcing powerful economic sanctions, cutting off Russia's largest banks from the international financial system, preventing Russia's central bank from defending the Russian Ruble, and choking off Russia's access to technology that will sap its economic stre